In [1]:
import cv2
import numpy as np

# 'path to input image/video'

# 'path to yolo config file' 
# download https://github.com/arunponnusamy/object-detection-opencv/blob/master/yolov3.cfg
CONFIG='./yolo/yolov3.cfg'

# 'path to text file containing class names'
# download https://github.com/arunponnusamy/object-detection-opencv/blob/master/yolov3.txt
CLASSES='./yolo/yolov3.txt'

# 'path to yolo pre-trained weights' 
# wget https://pjreddie.com/media/files/yolov3.weights
WEIGHTS='./yolo/yolov3.weights'

In [2]:
import os  
print(os.path.exists(CLASSES))
print(os.path.exists(CONFIG))
print(os.path.exists(WEIGHTS))

True
True
True


In [3]:
classes = None
with open(CLASSES, 'r') as f:
     classes = [line.strip() for line in f.readlines()]
        
scale = 0.00392
conf_threshold = 0.5
nms_threshold = 0.4

# generate different colors for different classes 
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

In [4]:
def get_output_layers(net): 
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers

# function to draw bounding box on the detected object with class name
def draw_bounding_box(img, class_id, confidence, x, y, x_plus_w, y_plus_h):
    label = str(classes[class_id])
    color = COLORS[class_id]
    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)
    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

In [5]:
def processImage(image, index):

    Width = image.shape[1]
    Height = image.shape[0]

    # read pre-trained model and config file
    net = cv2.dnn.readNet(WEIGHTS, CONFIG)

    # create input blob 
    blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)
    # set input blob for the network
    net.setInput(blob)

    # run inference through the network
    # and gather predictions from output layers
    outs = net.forward(get_output_layers(net))

    # initialization
    class_ids = []
    confidences = []
    boxes = []
    # for each detetion from each output layer 
    # get the confidence, class id, bounding box params
    # and ignore weak detections (confidence < 0.5)
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * Width)
                center_y = int(detection[1] * Height)
                w = int(detection[2] * Width)
                h = int(detection[3] * Height)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])
                    
    # apply non-max suppression
    
    classified_objects = []
    for i in class_ids:
        classified_objects.append(classes[i])
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    
    # go through the detections remaining
    # after nms and draw bounding box
    for i in indices:
        i = i[0]
        box = boxes[i]
        x = box[0]
        y = box[1]
        w = box[2]
        h = box[3]
    
        draw_bounding_box(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))
    
    # display output image    
    
    #cv2.imshow(out_image_name, image)
    # wait until any key is pressed
    #cv2.waitKey()
    # save output image to disk
    return classified_objects

In [6]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

directory = 'juliahengel'
pr_classes = {}
for filename in os.listdir(directory):
    if filename.endswith('.jpg'):
        print(directory + '/' + filename)
        temp = mpimg.imread(directory + '/' + filename)
        pr_classes[filename] = processImage(temp, filename)

juliahengel/11373951_547861015368709_27383197_n.jpg
juliahengel/11137861_368928393296562_586395092_n.jpg
juliahengel/59440893_320447451978479_14440433370112300_n.jpg
juliahengel/1516017_677498145628443_539786127_n.jpg
juliahengel/10517972_791820837518301_1077354569_n.jpg
juliahengel/11377651_979035225454202_1395787757_n.jpg
juliahengel/10809822_931130613574609_767223753_n.jpg
juliahengel/11111538_472759336209408_864946583_n.jpg
juliahengel/11265882_1581736328749792_1519340051_n.jpg
juliahengel/23417009_901911826642545_1995106574934212608_n.jpg
juliahengel/10895085_876160072429273_800102039_n.jpg
juliahengel/11939590_1514410338851042_1833829311_n.jpg
juliahengel/28754159_2009000082707409_1496938351972843520_n.jpg
juliahengel/12552255_431180823755321_1736729546_n.jpg
juliahengel/11917942_996161067094383_1704580930_n.jpg
juliahengel/22857619_948394388641845_4255404522580475904_n.jpg
juliahengel/13414214_144654869285817_161266266_n.jpg
juliahengel/26067697_948562815320572_38612859797635072

juliahengel/12724779_1684586471799611_1742057789_n.jpg
juliahengel/11189775_1611591242417060_1757952526_n.jpg
juliahengel/10246039_658726870831715_1504353742_n.jpg
juliahengel/12965836_846314465497184_1030855256_n.jpg
juliahengel/11380994_1074576022570986_1407304232_n.jpg
juliahengel/12063140_212144615784034_921354379_n.jpg
juliahengel/32156289_2054476221469954_79533710466088960_n.jpg
juliahengel/917153_789150444449908_1875588500_n.jpg
juliahengel/14714533_1127277440684017_1673070910627643392_n.jpg
juliahengel/11190736_568805359924113_1713093256_n.jpg
juliahengel/12353939_1096859176999067_163133888_n.jpg
juliahengel/12353938_1541185229540161_971137352_n.jpg
juliahengel/11254327_1634705056773237_578860528_n.jpg
juliahengel/13534549_1727491330852244_796338287_n.jpg
juliahengel/20590029_138818706710037_6432910215317815296_n.jpg
juliahengel/10249153_247286722124404_451390833_n.jpg
juliahengel/1516510_1394717670775827_798767730_n.jpg
juliahengel/10932350_1531127950485797_1251482691_n.jpg
ju

juliahengel/18722200_436438663391670_7743797726558552064_n.jpg
juliahengel/11208413_993057854038759_576741373_n.jpg
juliahengel/18013145_383523022047942_8669724119308173312_n.jpg
juliahengel/11093061_1388121131514080_899916669_n.jpg
juliahengel/15306698_1664880877144089_6550728349432414208_n.jpg
juliahengel/11246618_300764733430654_1067751027_n.jpg
juliahengel/16788460_260159631105047_6440932140484919296_n.jpg
juliahengel/11820452_1702901689940597_1621072132_n.jpg
juliahengel/928892_1426471020991479_209524123_n.jpg
juliahengel/51031611_828108167537715_1218216175752296409_n.jpg
juliahengel/12725086_1180378175336062_1260714737_n.jpg
juliahengel/15803568_388553871493234_7418626246499631104_n.jpg
juliahengel/11199384_1076571102360210_993393960_n.jpg
juliahengel/11379794_1013995958631151_1959739297_n.jpg
juliahengel/11287824_1593751294219499_913284436_n.jpg
juliahengel/928036_1537207313168145_513943795_n.jpg
juliahengel/14705102_1866957956922163_5039949224830042112_n.jpg
juliahengel/1242493

juliahengel/11330696_802510156485399_75473878_n.jpg
juliahengel/42578627_334679010640452_8552266250644459366_n.jpg
juliahengel/11201575_433878536780800_168173906_n.jpg
juliahengel/25006065_1995073447432517_7745611564262096896_n.jpg
juliahengel/11260565_825372820874825_120952474_n.jpg
juliahengel/11263347_1408877876102329_939827732_n.jpg
juliahengel/10724662_185577071612897_254425098_n.jpg
juliahengel/10838444_664420380343174_808276705_n.jpg
juliahengel/10787859_972836482727476_1055877557_n.jpg
juliahengel/11378853_1585954935019031_1747940340_n.jpg
juliahengel/10732042_1486899038242208_1257320895_n.jpg
juliahengel/55931168_107738420302866_4059185342273496880_n.jpg
juliahengel/11271969_641140496020376_1714748200_n.jpg
juliahengel/11191464_967705896574072_1666562525_n.jpg
juliahengel/11241393_817889944942860_1122884171_n.jpg
juliahengel/10584717_668311673255643_507485403_n.jpg
juliahengel/15043456_874671599336512_4092751486855938048_n.jpg
juliahengel/51944535_2443208202420523_728177961758

juliahengel/1208205_247636768724437_239782794_n.jpg
juliahengel/11176486_986573684687675_1299083570_n.jpg
juliahengel/10518152_1514755532072442_1112701009_n.jpg
juliahengel/23161000_1966228900331844_4877632424915238912_n.jpg
juliahengel/11327314_910346752362348_1407657717_n.jpg
juliahengel/10963957_1563699960583199_1154989772_n.jpg
juliahengel/11084742_883187158414712_1692621860_n.jpg
juliahengel/11184718_888221947910645_1733384798_n.jpg
juliahengel/11245871_484321165049408_1984956379_n.jpg
juliahengel/11176064_1547025765518029_884394481_n.jpg
juliahengel/11189380_492527817567728_88847649_n.jpg
juliahengel/11191213_358954374307204_801093702_n.jpg
juliahengel/45570616_320221365245319_2302595404117808492_n.jpg
juliahengel/12545416_215168345489828_787565896_n.jpg
juliahengel/35617346_810181225856906_6042569341050814464_n.jpg
juliahengel/11312381_1611283345815219_336387421_n.jpg
juliahengel/14099432_290062414699697_866234018_n.jpg
juliahengel/1171073_550219178401974_16747093_n.jpg
juliahen

...

juliahengel/16124019_404144656588839_5944435780788158464_n.jpg
juliahengel/12070917_1498815690412374_145954886_n.jpg
juliahengel/12976096_1533855870251010_827173399_n.jpg
juliahengel/11008262_220390728312111_1062582052_n.jpg
juliahengel/11191317_895046567183176_159351174_n.jpg
juliahengel/11236359_1608286969418321_1929830547_n.jpg
juliahengel/24177670_925047984338635_6201421631366103040_n.jpg
juliahengel/11189572_413150122198742_680085350_n.jpg
juliahengel/10631999_169936690037687_343315357_n.jpg
juliahengel/37726615_274216299851455_2049025736102117376_n.jpg
juliahengel/20837483_1476115029122795_8983331524476665856_n.jpg
juliahengel/10986146_963310780360390_490784786_n.jpg
juliahengel/37984959_629604247433132_4998634959452241920_n.jpg
juliahengel/11280167_1451059258525919_871845613_n.jpg
juliahengel/1391263_415456068580312_689446690_n.jpg
juliahengel/12798006_967727229931264_677550989_n.jpg
juliahengel/18380936_121354405102117_1923720593522294784_n.jpg
juliahengel/21909683_157055472632

juliahengel/11264166_1395812950746519_689533875_n.jpg
juliahengel/11374539_1025808020785274_1098363_n.jpg
juliahengel/11184479_861364017262912_1454400325_n.jpg
juliahengel/11311189_958061590911167_381938325_n.jpg
juliahengel/11078820_1411173825869125_591594976_n.jpg
juliahengel/11116843_896814670384698_1217677345_n.jpg
juliahengel/11385160_359257440942511_1028716698_n.jpg
juliahengel/11116661_381728818677811_1218280293_n.jpg
juliahengel/14310866_1589948934647804_582936410_n.jpg
juliahengel/11138002_1435152340113609_715630830_n.jpg
juliahengel/11142121_372405169612717_968211951_n.jpg
juliahengel/18095759_717555555112797_8843849914623459328_n.jpg
juliahengel/10914155_937175232970979_1622460720_n.jpg
juliahengel/37752871_2127381670846770_9078025765122998272_n.jpg
juliahengel/11116722_101843403481500_1490360993_n.jpg
juliahengel/1515138_1429058460664781_861951825_n.jpg
juliahengel/12818945_426180167579790_1839879211_n.jpg
juliahengel/11230487_104755096525552_2053677680_n.jpg
juliahengel/11

juliahengel/13116670_1725486331040764_1604746601_n.jpg
juliahengel/26867855_310562572799406_523884047588392960_n.jpg
juliahengel/10601681_688221737926180_2004721226_n.jpg
juliahengel/23595713_141268106597722_1177758458666024960_n.jpg
juliahengel/13414398_1453289314696815_124956136_n.jpg
juliahengel/11192700_814704455272009_2023795032_n.jpg
juliahengel/44835962_322643961868282_5232190329040357204_n.jpg
juliahengel/12729448_1556505924665662_736014205_n.jpg
juliahengel/12328123_1517742431863784_2013627502_n.jpg
juliahengel/22582083_1116449668457727_4491576824203051008_n.jpg
juliahengel/11374104_379881718885008_21253405_n.jpg
juliahengel/39938619_1511268252307073_1480569086906512896_n.jpg
juliahengel/11184706_367521623442765_431593326_n.jpg
juliahengel/11117198_328565887313626_1632619154_n.jpg
juliahengel/11910299_1455832558080797_1896081959_n.jpg
juliahengel/11190252_920288187992088_538204155_n.jpg
juliahengel/51426536_149915676005425_497087358878641322_n.jpg
juliahengel/11142830_65252171

juliahengel/10724829_157036551133243_1976462370_n.jpg
juliahengel/13473129_1120283738033269_1773366020_n.jpg
juliahengel/14287991_351999281854920_898698550_n.jpg
juliahengel/11184686_1389147574744763_975448490_n.jpg
juliahengel/11386506_1442283842743783_1291202766_n.jpg
juliahengel/33964079_2091327824477687_8704459856293134336_n.jpg
juliahengel/10986401_433709763454401_1507116477_n.jpg
juliahengel/12346034_606211579516977_586952045_n.jpg
juliahengel/924714_660791217383660_1780805375_n.jpg
juliahengel/11117051_1633810493516282_711892547_n.jpg
juliahengel/19120694_463613683998900_2005270563500589056_n.jpg
juliahengel/11189271_921242327937771_1700858413_n.jpg
juliahengel/12552209_1010763658967112_259264789_n.jpg
juliahengel/14727515_674867142689045_1621720200032288768_n.jpg
juliahengel/40958056_314675405782585_3149959594307318444_n.jpg
juliahengel/10387810_297334837097664_369934519_n.jpg
juliahengel/53894746_368757753973984_1622684540919440042_n.jpg
juliahengel/54511022_2373669806188453_6

KeyboardInterrupt: 

In [7]:
import csv
w = csv.writer(open("JuliaH_output.csv", "w"))
for key, val in pr_classes.items():
    w.writerow([key, val])